In [1]:
import numpy as np 
import pandas as pd 
import joblib
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import joblib
from sklearn.preprocessing import MinMaxScaler

torch.manual_seed(0)
np.random.seed(0)

calculate_loss_over_all_values = False

In [2]:
import importlib
login_str = 'login_haj'
module = importlib.import_module(login_str, package=None)
account_name = login_str
print('Logging in with account : '+str(account_name))
module.retry_autologin()

Logging in with account : login_haj
Trying to login...
access token : q0PRAtD3L2ukGNEsOBLddGjZxiF4H6qz
request token : tibQLvfPH8JauNqzvqXMbXf7UOGkSSPf


In [15]:
# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number
#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)
#print(out)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_window = 300
output_window = 5
batch_size = 10 # batch size

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransAm(nn.Module):
    def __init__(self,feature_size=30,num_layers=2,dropout=0.2):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()
    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)


def get_data2(inst):
    
    today_dt = datetime.datetime.now()
    today = f'{today_dt.day}-{today_dt.month}-{today_dt.year}'
    from_date_dt =  today_dt - datetime.timedelta(days=days_window)
    from_date = f'{from_date_dt.day}-{from_date_dt.month}-{from_date_dt.year}'

    interval = '5minute'
    new_lst = 0
    try:
        new_lst = module.kite.historical_data(inst, from_date_dt, today_dt, interval,continuous=False)
    except Exception as crr:
        print(crr)
            
    df_nifty = pd.DataFrame(new_lst)
    
    this_inst_df = df_nifty
    amplitude = this_inst_df['close'].to_numpy()
    amplitude = amplitude.reshape(-1)
    
    scaler = MinMaxScaler(feature_range=(-15, 15)) 
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    test_data = amplitude
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window]
    print(test_data.shape)
    
    return test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        
        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
plot_counter = 0
def plot_and_loss(eval_model, data_source,epoch):
    global plot_counter
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()

            test_result = torch.cat((test_result.to(device), output[-1].view(-1).to(device)), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth.to(device), target[-1].view(-1).to(device)), 0)
            
#             test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
#             truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    test_result = test_result.cpu().numpy()
    truth = truth.cpu().numpy()
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(f'./transformer-epoch_{plot_counter}_{epoch}.png')
    plot_counter +=1
    pyplot.close()
    
    return total_loss / i


def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
#     data = data.to(device).view(-1)
    data = data.cpu().view(-1)
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(f'./transformer-future_{plot_counter}_{steps}.png')
    pyplot.close()
        
# entweder ist hier ein fehler im loss oder in der train methode, aber die ergebnisse sind unterschiedlich 
# auch zu denen der predict_future
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).to(device).item()
#                 total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).to(device).item()            
#                 total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)



In [16]:
# input_window = 300
# output_window = 5
# batch_size = 10 # batch size

# def predict_future(eval_model, data_source,steps):
#     eval_model.eval() 
#     total_loss = 0.
#     test_result = torch.Tensor(0)    
#     truth = torch.Tensor(0)
#     _ , data = get_batch(data_source, 0,1)
#     with torch.no_grad():
#         for i in range(0, steps,1):
#             input = torch.clone(data[-input_window:])
#             input[-output_window:] = 0     
#             output = eval_model(data[-input_window:])                        
#             data = torch.cat((data, output[-1:]))
            
# #     data = data.to(device).view(-1)
#     data = data.cpu().view(-1)
#     pyplot.plot(data,color="red")       
#     pyplot.plot(data[:input_window],color="blue")
#     pyplot.grid(True, which='both')
#     pyplot.axhline(y=0, color='k')
#     pyplot.savefig(f'./transformer-future_{steps}.png')
#     pyplot.close()
        
# def get_batch(source, i,batch_size):
#     seq_len = min(batch_size, len(source) - 1 - i)
#     data = source[i:i+seq_len]    
#     input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
#     target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
#     return input, target

# def create_inout_sequences(input_data, tw):
#     inout_seq = []
#     L = len(input_data)
#     for i in range(L-tw):
#         train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
#         train_label = input_data[i:i+tw]
#         #train_label = input_data[i+output_window:i+tw+output_window]
#         inout_seq.append((train_seq ,train_label))
#     return torch.FloatTensor(inout_seq)

In [17]:
# nput_window = 300
# output_window = 5
# batch_size = 10 # batch size
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super(PositionalEncoding, self).__init__()       
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         #pe.requires_grad = False
#         self.register_buffer('pe', pe)
#     def forward(self, x):
#         return x + self.pe[:x.size(0), :]
       

# class TransAm(nn.Module):
#     def __init__(self,feature_size=30,num_layers=2,dropout=0.2):
#         super(TransAm, self).__init__()
#         self.model_type = 'Transformer'
#         self.src_mask = None
#         self.pos_encoder = PositionalEncoding(feature_size)
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
#         self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
#         self.decoder = nn.Linear(feature_size,1)
#         self.init_weights()
#     def init_weights(self):
#         initrange = 0.1    
#         self.decoder.bias.data.zero_()
#         self.decoder.weight.data.uniform_(-initrange, initrange)
#     def forward(self,src):
#         if self.src_mask is None or self.src_mask.size(0) != len(src):
#             device = src.device
#             mask = self._generate_square_subsequent_mask(len(src)).to(device)
#             self.src_mask = mask
#         src = self.pos_encoder(src)
#         output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
#         output = self.decoder(output)
#         return output
#     def _generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask


In [6]:
model= torch.load('./best_model_multi6.pt',map_location=torch.device('cpu'))

In [7]:
def pred_fut(days_window,insttkn,look_up):
    '''pred_fut(days_window,insttkn,look_up)'''

    today_dt = datetime.datetime.now()
    today = f'{today_dt.day}-{today_dt.month}-{today_dt.year}'
    from_date_dt =  today_dt - datetime.timedelta(days=days_window)
    from_date = f'{from_date_dt.day}-{from_date_dt.month}-{from_date_dt.year}'

    interval = '5minute'
    new_lst = 0
    try:
        new_lst = module.kite.historical_data(insttkn, from_date_dt, today_dt, interval,continuous=False)
    except Exception as crr:
        print(crr)

    df_nifty = pd.DataFrame(new_lst)
    # print(df_nifty)

    close = df_nifty['close']

    dates = df_nifty['date'][-look_up:].to_list()

    amplitude = close.to_numpy()
    amplitude = amplitude.reshape(-1)
    scaler = MinMaxScaler(feature_range=(-15, 15)) 
    trf = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

    hjhjhj = torch.tensor(trf[-look_up:].reshape(look_up,1,1)).float()
    # hjhjhj = torch.tensor(df_nifty['close'][-300:].to_numpy().reshape(300,1,1)).float()
    # print(hjhjhj)

    rp = model(hjhjhj).reshape(look_up,1)

    op = scaler.inverse_transform(rp.detach().numpy())
    # op = rp.detach().numpy()

    predicted = op[-look_up:]
    
    predicted_df = pd.DataFrame(predicted)
    predicted_df.plot(figsize=(9,5),title = str(insttkn))

In [13]:
def pred_fut2(days_window,insttkn,look_up):
    '''pred_fut2(days_window,insttkn,look_up)'''

#     today_dt = datetime.datetime.now()
#     today = f'{today_dt.day}-{today_dt.month}-{today_dt.year}'
#     from_date_dt =  today_dt - datetime.timedelta(days=days_window)
#     from_date = f'{from_date_dt.day}-{from_date_dt.month}-{from_date_dt.year}'

#     interval = '5minute'
#     new_lst = 0
#     try:
#         new_lst = module.kite.historical_data(insttkn, from_date_dt, today_dt, interval,continuous=False)
#     except Exception as crr:
#         print(crr)
            
#     df_nifty = pd.DataFrame(new_lst)
        
#     amplitude = df_nifty['close'].to_numpy().reshape(-1)
    
#     scaler = MinMaxScaler(feature_range=(-15, 15)) 
    
#     test_data = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
#     test_data = create_inout_sequences(test_data,input_window)
#     test_data = test_data[:-output_window].to(device)
    
#     print(test_data.shape)

    test_data_ip = get_data2(insttkn)
    print(test_data_ip.shape)
    
    rp = predict_future(model, test_data_ip,look_up)
    
    print(rp.shape)

#     rp = model(hjhjhj).reshape(look_up,1)

    op = scaler.inverse_transform(rp.detach().numpy())
    # op = rp.detach().numpy()

    predicted = op[-look_up:]
    
    predicted_df = pd.DataFrame(predicted)
    predicted_df.plot(figsize=(9,5),title = str(insttkn))

In [18]:
import datetime
days_window =60
insttkn = 103425
look_up = 200

inst_check_list = [633601,3930881,2029825,617473,98049,340481,6483969,5105409,]
# pred_fut(days_window,insttkn,look_up)

for one in inst_check_list:
    pred_fut2(days_window,one,look_up)

torch.Size([2695, 2, 300])
torch.Size([2695, 2, 300])


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
predict_future(model, data_source,steps)